In [104]:
import pandas as pd 
import numpy as np                     # For mathematical calculations 
import seaborn as sns                  # For data visualization 
import matplotlib.pyplot as plt        # For plotting graphs 
%matplotlib inline 
import warnings        

In [2]:
X=pd.read_csv('X.dummies.csv')

In [4]:
train=pd.read_csv('train.csv')

In [6]:
y=train.Loan_Status

In [35]:
X['Loan_Status']=y

In [39]:
x=X.iloc[:,1:]

In [144]:
test=pd.read_csv('testdummies.csv')

In [146]:
test=test.iloc[:,1:]

In [151]:
test=test.iloc[:,:-1]

In [152]:
test.shape

(367, 20)

In [61]:
x['Loan_Status'].replace('N', 0,inplace=True) 
x['Loan_Status'].replace('Y', 1,inplace=True)

In [62]:
x.head()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Gender_Female,Gender_Male,Married_No,Married_Yes,Dependents_3,...,Dependents_1,Dependents_2,Education_Graduate,Education_Not Graduate,Self_Employed_No,Self_Employed_Yes,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban,Loan_Status
0,5849,0.0,128.0,360.0,1.0,0,1,1,0,0,...,0,0,1,0,1,0,0,0,1,1
1,4583,1508.0,128.0,360.0,1.0,0,1,0,1,0,...,1,0,1,0,1,0,1,0,0,0
2,3000,0.0,66.0,360.0,1.0,0,1,0,1,0,...,0,0,1,0,0,1,0,0,1,1
3,2583,2358.0,120.0,360.0,1.0,0,1,0,1,0,...,0,0,0,1,1,0,0,0,1,1
4,6000,0.0,141.0,360.0,1.0,0,1,1,0,0,...,0,0,1,0,1,0,0,0,1,1


In [42]:
x_train=x.iloc[:,:-1]

In [43]:
x_train.head()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Gender_Female,Gender_Male,Married_No,Married_Yes,Dependents_3,Dependents_0,Dependents_1,Dependents_2,Education_Graduate,Education_Not Graduate,Self_Employed_No,Self_Employed_Yes,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban
0,5849,0.0,128.0,360.0,1.0,0,1,1,0,0,1,0,0,1,0,1,0,0,0,1
1,4583,1508.0,128.0,360.0,1.0,0,1,0,1,0,0,1,0,1,0,1,0,1,0,0
2,3000,0.0,66.0,360.0,1.0,0,1,0,1,0,1,0,0,1,0,0,1,0,0,1
3,2583,2358.0,120.0,360.0,1.0,0,1,0,1,0,1,0,0,0,1,1,0,0,0,1
4,6000,0.0,141.0,360.0,1.0,0,1,1,0,0,1,0,0,1,0,1,0,0,0,1


### ada boost 

In [44]:
from sklearn.ensemble import AdaBoostClassifier

In [45]:
from sklearn.tree import DecisionTreeClassifier

In [46]:
tree = DecisionTreeClassifier()
ada = AdaBoostClassifier( base_estimator=tree,random_state=1)

In [47]:
from sklearn.model_selection import GridSearchCV

In [48]:

n_estimators = [int(x) for x in np.linspace(start = 20, stop = 200, num = 10)]
learning_rate= [0.001,0.01,0.03,0.06,0.1,0.3,0.6,1,3,10.0,]


grid = {'n_estimators': n_estimators,
               'learning_rate':learning_rate,
               }
clf_ada = GridSearchCV(estimator = ada, param_grid = grid, 
                          cv = 5, n_jobs = -1)

In [49]:
best_ada=clf_ada.fit(x_train,y)

In [50]:
print(best_ada.best_score_)

0.6986970684039088


In [51]:
print(best_ada.best_params_)

{'learning_rate': 0.001, 'n_estimators': 20}


### xgboost

In [33]:
import xgboost as xgb
from xgboost.sklearn import XGBClassifier


In [68]:
from sklearn import metrics

In [53]:
target = 'Loan_Status';

In [108]:
def modelfit(alg, dtrain, predictors,useTrainCV=True, cv_folds=5, early_stopping_rounds=50):
    
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(dtrain[predictors].values, label=dtrain[target].values)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
            metrics='error', early_stopping_rounds=early_stopping_rounds,verbose_eval=True)
        alg.set_params(n_estimators=cvresult.shape[0])
        print(cvresult)
        print(cvresult.shape[0])
    #Fit the algorithm on the data
    alg.fit(dtrain[predictors], dtrain['Loan_Status'],eval_metric='error')
        
    #Predict training set:
    dtrain_predictions = alg.predict(dtrain[predictors])
    dtrain_predprob = alg.predict_proba(dtrain[predictors])[:,1]
        
    #Print model report:
    print ("\nModel Report")
    print ("Accuracy : %.4g" % metrics.accuracy_score(dtrain['Loan_Status'].values, dtrain_predictions))
    print ("AUC Score (Train): %f" % metrics.roc_auc_score(dtrain['Loan_Status'], dtrain_predprob))
                    
  #  feat_imp = pd.Series(alg.booster().get_fscore()).sort_values(ascending=False)
   # feat_imp.plot(kind='bar', title='Feature Importances')
   # plt.ylabel('Feature Importance Score')

In [109]:
#Choose all predictors except target & IDcols
predictors = [y for y in x_train.columns ]
xgb1 = XGBClassifier(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 njobs=-1,
 scale_pos_weight=1,
 random_state=1)
modelfit(xgb1, x, predictors)

[0]	train-error:0.174672+0.00576332	test-error:0.224777+0.0211727
[1]	train-error:0.174673+0.00320718	test-error:0.197095+0.0250857
[2]	train-error:0.180783+0.00598042	test-error:0.206851+0.0497852
[3]	train-error:0.175898+0.00717437	test-error:0.201986+0.0256611
[4]	train-error:0.175083+0.00622468	test-error:0.201959+0.0396487
[5]	train-error:0.173452+0.00720322	test-error:0.197095+0.0197824
[6]	train-error:0.171417+0.00649777	test-error:0.198734+0.0287185
[7]	train-error:0.171417+0.00788225	test-error:0.197081+0.0215595
[8]	train-error:0.170196+0.00811734	test-error:0.193829+0.0203495
[9]	train-error:0.171823+0.00675589	test-error:0.192203+0.0191728
[10]	train-error:0.172231+0.00597851	test-error:0.197081+0.0221642
[11]	train-error:0.16938+0.00562932	test-error:0.195469+0.0214758
[12]	train-error:0.171823+0.00594935	test-error:0.195469+0.0214758
[13]	train-error:0.16938+0.00621439	test-error:0.193843+0.0191614
[14]	train-error:0.169788+0.00652727	test-error:0.193843+0.0191614
[15]	tr

In [110]:
param_test1 = {
 'max_depth':range(3,10,2),
 'min_child_weight':range(1,6,2)
}
gsearch1 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=27, max_depth=5,
 min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', njobs=-1, scale_pos_weight=1, random_state=1), 
 param_grid = param_test1,n_jobs=-1,iid=False, cv=5)
gsearch1.fit(x[predictors],x[target])
gsearch1.best_params_, gsearch1.best_score_

({'max_depth': 3, 'min_child_weight': 3}, 0.8112109994711793)

In [111]:
param_test2 = {
 'max_depth':[2,3,4],
 'min_child_weight':[2,3,4]
}
gsearch2 = GridSearchCV(estimator = XGBClassifier( learning_rate=0.1, n_estimators=27, max_depth=5,
 min_child_weight=2, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', njobs=-1, scale_pos_weight=1,seed=1), 
 param_grid = param_test2,n_jobs=-1,iid=False, cv=5)
gsearch2.fit(x[predictors],x[target])
gsearch2.best_params_, gsearch2.best_score_

({'max_depth': 3, 'min_child_weight': 2}, 0.8112109994711793)

In [112]:
param_test2b = {
 'min_child_weight':[1,2,3]
}
gsearch2b = GridSearchCV(estimator = XGBClassifier( learning_rate=0.1, n_estimators=27, max_depth=3,
 min_child_weight=2, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', njobs=-1, scale_pos_weight=1,seed=1), 
 param_grid = param_test2b,n_jobs=-1,iid=False, cv=5)
gsearch2b.fit(x[predictors],x[target])
gsearch2b.best_params_, gsearch2b.best_score_

({'min_child_weight': 2}, 0.8112109994711793)

In [113]:
param_test3 = {
 'gamma':[i/10.0 for i in range(0,5)]
}
gsearch3 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=27, max_depth=3,
 min_child_weight=2, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', njobs=-1, scale_pos_weight=1,seed=1), 
 param_grid = param_test3,n_jobs=-1,iid=False, cv=5)
gsearch3.fit(x[predictors],x[target])
gsearch3.best_params_, gsearch3.best_score_

({'gamma': 0.0}, 0.8112109994711793)

In [114]:
xgb2 = XGBClassifier(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=3,
 min_child_weight=2,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 n_jobs=-1,
 scale_pos_weight=1,
 seed=1)
modelfit(xgb2, x, predictors)

[0]	train-error:0.190963+0.00640822	test-error:0.203585+0.0272167
[1]	train-error:0.187704+0.0043589	test-error:0.193829+0.0239319
[2]	train-error:0.194221+0.00708436	test-error:0.198707+0.0328197
[3]	train-error:0.188111+0.00458867	test-error:0.190577+0.0184998
[4]	train-error:0.192999+0.00313858	test-error:0.193829+0.0239319
[5]	train-error:0.189333+0.00468006	test-error:0.190577+0.0184998
[6]	train-error:0.189333+0.00468006	test-error:0.190577+0.0184998
[7]	train-error:0.189333+0.00468006	test-error:0.190577+0.0184998
[8]	train-error:0.188926+0.00456569	test-error:0.190577+0.0184998
[9]	train-error:0.188926+0.00456569	test-error:0.190577+0.0184998
[10]	train-error:0.188519+0.00421858	test-error:0.190577+0.0184998
[11]	train-error:0.188519+0.00421858	test-error:0.190577+0.0184998
[12]	train-error:0.188519+0.00421858	test-error:0.190577+0.0184998
[13]	train-error:0.188519+0.00421858	test-error:0.190577+0.0184998
[14]	train-error:0.188111+0.00421124	test-error:0.190577+0.0184998
[15]	t


Model Report
Accuracy : 0.816
AUC Score (Train): 0.907608


In [115]:
param_test4 = {
 'subsample':[i/10.0 for i in range(6,10)],
 'colsample_bytree':[i/10.0 for i in range(6,10)]
}
gsearch4 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=27, max_depth=3,
 min_child_weight=2, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=-1, scale_pos_weight=1,seed=1), 
 param_grid = param_test4, n_jobs=-1,iid=False, cv=5)
gsearch4.fit(x[predictors],x[target])
gsearch4.best_params_, gsearch4.best_score_

({'colsample_bytree': 0.9, 'subsample': 0.7}, 0.8128239026969858)

In [117]:
param_test5 = {
 'subsample':[i/100.0 for i in range(65,80,5)],
 'colsample_bytree':[i/100.0 for i in range(85,100,5)]
}
gsearch5 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=27, max_depth=3,
 min_child_weight=2, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=-1, scale_pos_weight=1,seed=1), 
 param_grid = param_test5,n_jobs=-1,iid=False, cv=5)
gsearch5.fit(x[predictors],x[target])
gsearch5.best_params_, gsearch5.best_score_

({'colsample_bytree': 0.9, 'subsample': 0.7}, 0.8128239026969858)

In [119]:
param_test6 = {
 'reg_alpha':[1e-5, 1e-2,0, 0.1, 1, 100]
}
gsearch6 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=27, max_depth=3,
 min_child_weight=2, gamma=0.0, subsample=0.7, colsample_bytree=0.9,
 objective= 'binary:logistic', nthread=-1, scale_pos_weight=1,seed=1), 
 param_grid = param_test6, n_jobs=-1,iid=False, cv=5)
gsearch6.fit(x[predictors],x[target])
gsearch6.best_params_, gsearch6.best_score_

({'reg_alpha': 1e-05}, 0.8128239026969858)

In [127]:
param_test7 = {
 'reg_alpha':[0.0075, 0.007,0.0067, 0.0065, 0.006]
}
gsearch7 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=27, max_depth=3,
 min_child_weight=2, gamma=0.0, subsample=0.7, colsample_bytree=0.9,
 objective= 'binary:logistic', nthread=-1, scale_pos_weight=1,seed=1), 
 param_grid = param_test7, n_jobs=-1,iid=False, cv=5)
gsearch7.fit(x[predictors],x[target])
gsearch7.best_params_, gsearch7.best_score_

({'reg_alpha': 0.0075}, 0.8128239026969858)

In [128]:
xgb3 = XGBClassifier(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=3,
 min_child_weight=2,
 gamma=0,
 subsample=0.7,
 colsample_bytree=0.9,
 reg_alpha=0.0075,
 objective= 'binary:logistic',
 nthread=-1,
 scale_pos_weight=1,
 seed=1)
modelfit(xgb3, x, predictors)

[0]	train-error:0.190144+0.00602851	test-error:0.205211+0.0264076
[1]	train-error:0.190148+0.00496822	test-error:0.192203+0.0177403
[2]	train-error:0.190555+0.00384555	test-error:0.190577+0.0184998
[3]	train-error:0.190148+0.00559623	test-error:0.190577+0.0184998
[4]	train-error:0.189741+0.00509369	test-error:0.190577+0.0184998
[5]	train-error:0.189741+0.00509369	test-error:0.190577+0.0184998
[6]	train-error:0.189333+0.00502195	test-error:0.190577+0.0184998
[7]	train-error:0.189741+0.00509369	test-error:0.190577+0.0184998
[8]	train-error:0.189333+0.00368894	test-error:0.190577+0.0184998
[9]	train-error:0.189333+0.00411404	test-error:0.190577+0.0184998
[10]	train-error:0.189333+0.00411404	test-error:0.190577+0.0184998
[11]	train-error:0.188518+0.00334083	test-error:0.190577+0.0184998
[12]	train-error:0.188518+0.0045949	test-error:0.190577+0.0184998
[13]	train-error:0.188518+0.0045949	test-error:0.190577+0.0184998
[14]	train-error:0.188111+0.00400937	test-error:0.190577+0.0184998
[15]	tr

In [132]:
param_test8 = {
 'scale_pos_weight':[0.2,0.4,0.6,0.8,1.0]
}
gsearch8 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=34, max_depth=3,
 min_child_weight=2, gamma=0.0, subsample=0.7, colsample_bytree=0.9,
 objective= 'binary:logistic', nthread=-1, scale_pos_weight=1,seed=1,reg_alpha=0.0075), 
 param_grid = param_test8, n_jobs=-1,iid=False, cv=5)
gsearch8.fit(x[predictors],x[target])
gsearch8.best_params_, gsearch8.best_score_

({'scale_pos_weight': 0.8}, 0.8128239026969858)

In [133]:
param_test9 = {
 'scale_pos_weight':[0.7,0.75,0.8,0.85,0.9]
}
gsearch9 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=34, max_depth=3,
 min_child_weight=2, gamma=0.0, subsample=0.7, colsample_bytree=0.9,
 objective= 'binary:logistic', nthread=-1, scale_pos_weight=1,seed=1,reg_alpha=0.0075), 
 param_grid = param_test9, n_jobs=-1,iid=False, cv=5)
gsearch9.fit(x[predictors],x[target])
gsearch9.best_params_, gsearch9.best_score_

({'scale_pos_weight': 0.75}, 0.8144632469592809)

In [134]:
xgb3 = XGBClassifier(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=3,
 min_child_weight=2,
 gamma=0,
 subsample=0.7,
 colsample_bytree=0.9,
 reg_alpha=0.0075,
 objective= 'binary:logistic',
 nthread=-1,
 scale_pos_weight=0.75,
 seed=1)
modelfit(xgb3, x, predictors)

[0]	train-error:0.198288+0.0123608	test-error:0.226376+0.026838
[1]	train-error:0.189335+0.00508136	test-error:0.201973+0.0216166
[2]	train-error:0.189334+0.00453293	test-error:0.190564+0.0203954
[3]	train-error:0.188113+0.0060488	test-error:0.192217+0.0200003
[4]	train-error:0.18852+0.00562767	test-error:0.188938+0.017492
[5]	train-error:0.188112+0.00588489	test-error:0.188938+0.017492
[6]	train-error:0.188111+0.00556847	test-error:0.190577+0.0184998
[7]	train-error:0.186891+0.00477511	test-error:0.190577+0.0184998
[8]	train-error:0.186077+0.00531847	test-error:0.190577+0.0184998
[9]	train-error:0.184854+0.00474337	test-error:0.190577+0.0184998
[10]	train-error:0.186075+0.00421118	test-error:0.190577+0.0184998
[11]	train-error:0.186482+0.00397841	test-error:0.190577+0.0184998
[12]	train-error:0.186075+0.00523281	test-error:0.188951+0.0160848
[13]	train-error:0.186482+0.00397841	test-error:0.190577+0.0184998
[14]	train-error:0.185261+0.00427494	test-error:0.192203+0.0177403
[15]	train-

In [135]:
xgb4 = XGBClassifier(
 learning_rate =0.01,
 n_estimators=5000,
 max_depth=3,
 min_child_weight=2,
 gamma=0,
 subsample=0.7,
 colsample_bytree=0.9,
 reg_alpha=0.0075,
 objective= 'binary:logistic',
 nthread=-1,
 scale_pos_weight=0.75,
 seed=1)
modelfit(xgb4, x, predictors)

[0]	train-error:0.198288+0.0123608	test-error:0.226376+0.026838
[1]	train-error:0.190557+0.00469258	test-error:0.197095+0.0234516
[2]	train-error:0.188927+0.0033459	test-error:0.188938+0.017492
[3]	train-error:0.189335+0.00598119	test-error:0.192217+0.0200003
[4]	train-error:0.190147+0.00493683	test-error:0.190577+0.0184998
[5]	train-error:0.188521+0.00565371	test-error:0.188938+0.017492
[6]	train-error:0.190148+0.004439	test-error:0.190577+0.0184998
[7]	train-error:0.189741+0.00478877	test-error:0.190577+0.0184998
[8]	train-error:0.18974+0.00439442	test-error:0.190577+0.0184998
[9]	train-error:0.188926+0.00418649	test-error:0.190577+0.0184998
[10]	train-error:0.188926+0.00418649	test-error:0.190577+0.0184998
[11]	train-error:0.187296+0.00427479	test-error:0.190577+0.0184998
[12]	train-error:0.188926+0.00418649	test-error:0.190577+0.0184998
[13]	train-error:0.187704+0.00416418	test-error:0.190577+0.0184998
[14]	train-error:0.188518+0.00441087	test-error:0.190577+0.0184998
[15]	train-er

In [136]:
xgb4 = XGBClassifier(
 learning_rate =0.05,
 n_estimators=3000,
 max_depth=3,
 min_child_weight=2,
 gamma=0,
 subsample=0.7,
 colsample_bytree=0.9,
 reg_alpha=0.0075,
 objective= 'binary:logistic',
 nthread=-1,
 scale_pos_weight=0.75,
 seed=1)
modelfit(xgb4, x, predictors)

[0]	train-error:0.198288+0.0123608	test-error:0.226376+0.026838
[1]	train-error:0.190557+0.00469258	test-error:0.197095+0.0234516
[2]	train-error:0.189741+0.0040366	test-error:0.188938+0.017492
[3]	train-error:0.189335+0.00524213	test-error:0.192217+0.0200003
[4]	train-error:0.190554+0.00421905	test-error:0.190577+0.0184998
[5]	train-error:0.189334+0.00471245	test-error:0.188938+0.017492
[6]	train-error:0.18974+0.00439442	test-error:0.190577+0.0184998
[7]	train-error:0.190147+0.00421161	test-error:0.190577+0.0184998
[8]	train-error:0.189333+0.00368894	test-error:0.190577+0.0184998
[9]	train-error:0.188111+0.00421121	test-error:0.188938+0.017492
[10]	train-error:0.187704+0.00351631	test-error:0.188938+0.017492
[11]	train-error:0.187297+0.00368844	test-error:0.192203+0.0177403
[12]	train-error:0.188111+0.00421121	test-error:0.192203+0.0177403
[13]	train-error:0.187704+0.00351631	test-error:0.192203+0.0177403
[14]	train-error:0.188111+0.00333153	test-error:0.190577+0.0184998
[15]	train-er

In [137]:
xgb4 = XGBClassifier(
 learning_rate =0.2,
 n_estimators=1000,
 max_depth=3,
 min_child_weight=2,
 gamma=0,
 subsample=0.7,
 colsample_bytree=0.9,
 reg_alpha=0.0075,
 objective= 'binary:logistic',
 nthread=-1,
 scale_pos_weight=0.75,
 seed=1)
modelfit(xgb4, x, predictors)

[0]	train-error:0.198288+0.0123608	test-error:0.226376+0.026838
[1]	train-error:0.189742+0.00481933	test-error:0.201973+0.0216166
[2]	train-error:0.188927+0.00497634	test-error:0.193843+0.0223462
[3]	train-error:0.187296+0.00528372	test-error:0.195455+0.0147435
[4]	train-error:0.187296+0.00669502	test-error:0.190577+0.0153781
[5]	train-error:0.183635+0.00603892	test-error:0.193829+0.0167901
[6]	train-error:0.186075+0.00421118	test-error:0.193829+0.0167901
[7]	train-error:0.183227+0.00597862	test-error:0.190564+0.0168458
[8]	train-error:0.18282+0.00680483	test-error:0.188938+0.017492
[9]	train-error:0.182005+0.00387235	test-error:0.190591+0.0155726
[10]	train-error:0.182005+0.00482597	test-error:0.190577+0.0125367
[11]	train-error:0.17834+0.00219051	test-error:0.187312+0.00764765
[12]	train-error:0.177117+0.00129603	test-error:0.190564+0.0168458
[13]	train-error:0.177117+0.00129603	test-error:0.19219+0.0168149
[14]	train-error:0.175895+0.00352928	test-error:0.188938+0.0121383
[15]	train

In [138]:
xgb4 = XGBClassifier(
 learning_rate =0.3,
 n_estimators=1000,
 max_depth=3,
 min_child_weight=2,
 gamma=0,
 subsample=0.7,
 colsample_bytree=0.9,
 reg_alpha=0.0075,
 objective= 'binary:logistic',
 nthread=-1,
 scale_pos_weight=0.75,
 seed=1)
modelfit(xgb4, x, predictors)

[0]	train-error:0.198288+0.0123608	test-error:0.226376+0.026838
[1]	train-error:0.190555+0.00425522	test-error:0.201973+0.0203568
[2]	train-error:0.18811+0.00611014	test-error:0.187299+0.0170647
[3]	train-error:0.187702+0.00618703	test-error:0.197081+0.017498
[4]	train-error:0.187296+0.00669502	test-error:0.193829+0.0175598
[5]	train-error:0.18567+0.00641627	test-error:0.19219+0.0168149
[6]	train-error:0.186078+0.00659574	test-error:0.197068+0.0208078
[7]	train-error:0.179563+0.00750695	test-error:0.193843+0.0234996
[8]	train-error:0.179153+0.00883204	test-error:0.195429+0.0217092
[9]	train-error:0.177933+0.00749491	test-error:0.198667+0.0300613
[10]	train-error:0.175488+0.00535302	test-error:0.203572+0.0261269
[11]	train-error:0.173046+0.00693779	test-error:0.198694+0.018206
[12]	train-error:0.168973+0.00641994	test-error:0.198681+0.0187873
[13]	train-error:0.166934+0.0078489	test-error:0.20845+0.0148143
[14]	train-error:0.165717+0.00493459	test-error:0.213328+0.0135873
[15]	train-err

In [140]:
xgbfinal = XGBClassifier(
 learning_rate =0.2,
 n_estimators=1000,
 max_depth=3,
 min_child_weight=2,
 gamma=0,
 subsample=0.7,
 colsample_bytree=0.9,
 reg_alpha=0.0075,
 objective= 'binary:logistic',
 nthread=-1,
 scale_pos_weight=0.75,
 seed=1)

xgbfinal.fit(x[predictors], x['Loan_Status'],eval_metric='error')
        

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.9, gamma=0, learning_rate=0.2, max_delta_step=0,
       max_depth=3, min_child_weight=2, missing=None, n_estimators=1000,
       n_jobs=1, nthread=-1, objective='binary:logistic', random_state=0,
       reg_alpha=0.0075, reg_lambda=1, scale_pos_weight=0.75, seed=1,
       silent=True, subsample=0.7)

In [153]:
pred_xgb = xgbfinal.predict(test)

In [155]:
test_original=pd.read_csv('test_original.csv')
submission5=pd.read_csv("Sample_Submission.csv")
submission5['Loan_Status']=pred_xgb 
submission5['Loan_ID']=test_original['Loan_ID']
submission5['Loan_Status'].replace(0, 'N',inplace=True) 
submission5['Loan_Status'].replace(1, 'Y',inplace=True)
pd.DataFrame(submission5, columns=['Loan_ID','Loan_Status']).to_csv('xgb.csv')

Poor results than logistic itself.